# Generator Danych Testowych

## Import modułów i bibliotek

Skrypt wykorzystuje następujące moduły i biblioteki:
- [random](https://docs.python.org/3/library/random.html),
- [pandas](https://pandas.pydata.org/docs/),
- [Faker](https://faker.readthedocs.io/en/master/).

In [2]:
from random import random, uniform

import pandas as pd
from faker import Faker

fake = Faker("pl_PL")

## Podstawowe ustawienia

W słowniku `city` można ustawić nazwę (`name`) gminy, dla której chcemy wygenerować dane oraz zasięg (`y_min_max` oraz `x_min_max`), w którym mają zostać wygenerowane współrzędne punktowe. Zasięg można obliczyć w QGIS na podstawie granic gminy. Dane dla każdego z wymiarów podawane się w krotce (tuple), w której pierwsza liczba to minimum a druga maksimum.

Krotka `sample_categories` zawiera nazwy kategorii, które będą wykorzystane przy generowaniu danych.

Krotka `sample_sources` zawiera nazwy funduszy i programów, które będą wykorzystane przy generowaniu źródeł dofinansowania.

Zmienna `desired_samples` określa liczbę próbek, które mają zostać wygenerowane. **NB.** Należy wpisać 50%-60% więcej niż ma się faktycznie znaleźć w granicach. Po wygenrowaniu pliku należy go przyciąć do granic w QGIS.

Zmienna `dev_mode` to flaga. Jeżeli ustawiona jako prawda (`True`), plik .csv nie zostanie zapisany, w przeciwnym wypadku (`False`) plik zostanie zapisany.

In [2]:
city: dict[str, str | tuple[float | int, float | int]] = {
    "name": "Siemianowice Śląskie",
    "y_min_max": (50.274825, 50.349919),
    "x_min_max": (18.98523, 19.061271),
}

sample_categories: tuple[str, ...] = (
    "administracja",
    "dostępność",
    "infrastruktura",
    "kultura, sport i rekreacja",
    "ochrona środowiska",
    "oświata",
    "transport",
)

sample_sources: tuple[str, ...] = (
    "Fundusz Rozwoju Dróg Zwijanych",
    "Program Ochrony Wód przed Rybami",
    "Fundusz Ochrony Zabytków i Zabitków",
    "Fundusz Solidności Wykonania Prac Wszelakich",
    "Wojewódzki Program Naprawy do Poprawy",
    "Fundusz Poprawy Efektywności Efermerycznej",
    "Krajowy Program Ochrony przed Skutkami Wdrożeń",
    "Fundusz Mądrej Rozbudowy i Głupiej Zabudowy",
    "Program Wsparcia Administracji w Akcji",
    "Cyfryzacja+ - Państwowy Plan Zakupu Liczydeł",
)

desired_samples: int = 200

dev_mode: bool = False

## Generowanie danych

Funkcja `generate_empty()` zwraca listę pustych elementów. Wykorzystywana dla kolumn, które należy wypełnić na dalszym etapie prac.

Funkcja `generate_coords()` zwraca listę losowych współrzędnych w podanym zasięgu - domyślnie zasięg określony w `city["y_min_max"]` i `city["x_min_max"]`:
- współrzędne y - `generate_coords("y")`;
- współrzędne x - `generate_coords("x")`.

Funkcja `generate_pseudo_text()` zwraca listę bezsensownych tekstów o zadanej długości - domyślnie bez kropki na końcu. Aby zostawić kropkę na końcu tekstu, funkcję należy wywołać z parametrem `no_dot_at_end=False`. Długość generowanego tekstu ustawiana jest przy pomocy parametru `max_num_chars`. **NB.** wtyczka qgis2web może [automatycznie nie wygenerować obiektów, w których długość pola tekstowego przekracza 256 znaków](https://github.com/qgis2web/qgis2web/issues/947).

Funkcja `generate_from_sequence()` zwraca listę tekstów wybranych losowo z krotki.

Funkcja `generate_address_parts()` zwraca listę losowych części adresu. Możliwe części adresu to:
- cecha - `generate_address_parts("prefix")`;
- nazwa ulicy - `generate_address_parts("street")`;
- numer budynku - `generate_address_parts("building")`. W ok. 2/3 przypadków będzie to brak numeru budynku - jest to prawidłowe zachowanie;
- kod pocztowy - `generate_address_parts("postcode")`;
- nazwę miejscowości - `generate_address_parts("city")`. Domyślnie jest to nazwa miejscowości określona w `city["name"]`. W przypadku, gdy potrzebne są różne nazwy miejscowości, funkcję należy wywołać z parametrem `single_city=False`;
- w każdym innym przypadku zwracana jest lista pustych elementów.

Funkcja `generate_dates()` zwraca listę losowych dat:
- lata - `generate_dates("year")`;
- pełne daty - `generate_dates("full")`;
- w każdym innym przypadku zwracana jest lista pustych elementów.

Funkcja `generate_amounts()` zwraca listę losowych liczb zmiennoprzecinkowych z dwoma miejscami po przecinku. Liczbę cyfr przed przecinkiem można dostosować przy pomocy parametru (`num_left_digits`) - domyślnie 7.

Funkcja `generate_bools()` zwraca listę składającą się z losowo wybranych wartości prawda (`True`) lub fałsz (`False`).

Funkcja `generate_urls()` zwraca listę losowych adresów internetowych. W ok. 1/4 przypadków będzie to brak adresu internetowego - jest to prawidłowe zachowanie.

In [3]:
def generate_empty(num_to_generate: int = desired_samples) -> list[None]:
    return [None] * num_to_generate


def generate_coords(
    y_or_x: str,
    num_to_generate: int = desired_samples,
    extent_y: tuple[float | int, float | int] = city["y_min_max"],
    extent_x: tuple[float | int, float | int] = city["x_min_max"],
) -> list:
    coords: list = []
    match y_or_x:
        case "y":
            min_y, max_y = extent_y
            for _ in range(num_to_generate):
                coords.append(uniform(min_y, max_y))
        case "x":
            min_x, max_x = extent_x
            for _ in range(num_to_generate):
                coords.append(uniform(min_x, max_x))
        case _:
            coords = generate_empty(num_to_generate)
    return coords


def generate_pseudo_texts(
    num_to_generate: int = desired_samples,
    max_num_chars: int = 256,
    no_dot_at_end: bool = True,
) -> list[str]:
    texts: list[str] = fake.texts(nb_texts=num_to_generate, max_nb_chars=max_num_chars)
    if no_dot_at_end:
        for i in range(len(texts)):
            texts[i] = texts[i].rstrip(".")
    return texts


def generate_from_sequence(
    sequence: tuple, num_to_generate: int = desired_samples
) -> list:
    return fake.random_choices(elements=sequence, length=num_to_generate)


def generate_address_parts(
    address_part: str, num_to_generate: int = desired_samples, single_city: bool = True
) -> list:
    parts: list = []
    match address_part:
        case "prefix":
            parts = generate_from_sequence(
                ("ul.", "al.", "pl."), num_to_generate=num_to_generate
            )
        case "street":
            for _ in range(num_to_generate):
                parts.append(fake.street_name())
        case "building":
            for _ in range(num_to_generate):
                if random() > 0.33:
                    parts.append(fake.building_number())
                else:
                    parts.append(None)
        case "postcode":
            for _ in range(num_to_generate):
                parts.append(fake.postcode())
        case "city":
            for _ in range(num_to_generate):
                if single_city:
                    parts.append(city["name"])
                else:
                    parts.append(fake.city())
        case _:
            parts = generate_empty(num_to_generate)
    return parts


def generate_dates(year_or_full: str, num_to_generate: int = desired_samples) -> list:
    dates: list = []
    match year_or_full:
        case "year":
            for _ in range(num_to_generate):
                dates.append(fake.year())
        case "full":
            for _ in range(num_to_generate):
                dates.append(fake.date_between(start_date="-10y", end_date="-6m"))
        case _:
            dates = generate_empty(num_to_generate)
    return dates


def generate_amounts(
    num_left_digits: int = 7, num_to_generate: int = desired_samples
) -> list[float]:
    amounts: list = []
    for _ in range(num_to_generate):
        amounts.append(
            fake.pyfloat(left_digits=num_left_digits, right_digits=2, positive=True)
        )
    return amounts


def generate_bools(num_to_generate: int = desired_samples) -> list[bool]:
    bools: list[bool] = []
    for _ in range(num_to_generate):
        bools.append(fake.pybool())
    return bools


def generate_urls(num_to_generate: int = desired_samples) -> list[str | None]:
    urls: list[str | None] = []
    domain_name: str = fake.safe_domain_name()
    for _ in range(num_to_generate):
        if random() > 0.25:
            urls.append(f"https://{domain_name}/{fake.slug()}")
        else:
            urls.append(None)
    return urls

### Struktura tabeli

Słownik `d` zawiera strukturę generowanej tabeli. Każdy element to kolumna tabeli.

In [4]:
d = {
    # To be filled in in QGIS with image path:
    "foto_sciezka": generate_empty(),
    "kategoria": generate_from_sequence(sample_categories),
    "nazwa": generate_pseudo_texts(max_num_chars=64, no_dot_at_end=True),
    "zakres": generate_pseudo_texts(max_num_chars=256, no_dot_at_end=False),
    "cecha": generate_address_parts("prefix"),
    "ulica_nazwa": generate_address_parts("street"),
    # Will be empty for approx 1/3 samples, this is expected behaviour:
    "nr_budynku": generate_address_parts("building"),
    "kod_pocztowy": generate_address_parts("postcode"),
    # If multiple city names needed, change `single_city` to False:
    "miejscowosc": generate_address_parts("city", single_city=True),
    # To be filled in at the data cleaning stage:
    "adres": generate_empty(),
    "data_rozpoczecia": generate_dates("full"),
    # To be cleaned & adjusted at the data cleaning stage:
    "data_zakonczenia": generate_dates("full"),
    "czy_zakonczony": generate_bools(),
    "wartosc": generate_amounts(num_left_digits=7),
    "czy_dofinansowany": generate_bools(),
    # To be cleaned at the data cleaning stage:
    "zrodlo_dofinansowania": generate_from_sequence(sample_sources),
    # To be cleaned & adjusted at the data cleaning stage:
    "wartosc_dofinansowania": generate_amounts(),
    "czy_bo": generate_bools(),
    # To be cleaned at the data cleaning stage:
    "edycja_bo": generate_dates("year"),
    # Will be empty for approx 1/4 samples, this is expected behaviour:
    "url_fiszka": generate_urls(),
    # Will be empty for approx 1/4 samples, this is expected behaviour:
    "url_geoportal": generate_urls(),
    # Will be empty for approx 1/4 samples, this is expected behaviour
    "url_mapa": generate_urls(),
    # To be filled in at the data cleaning stage:
    "popup_url_html": generate_empty(),
    "y": generate_coords("y"),
    "x": generate_coords("x"),
}

## Oczyszczanie danych testowych

Wygenerowane dane wymagają oczyszczenie i uzupełnień przed eksportem.

Funkcja `fill_address_column()` populuje kolumnę `adres` na podstawie wygenerowanych danych cząstkowych.

Funkcja `adjust_aid_received()` dostosowuje wartość dofinansowania.

Funkcja `adjust_end_dates()` dostosowuje daty zakończenia.

Funkcja `fill_popup_url_html()` generuje kod html dla linków, który będzie wyświetlony w popupie.

Funkcja `clean_unwanted_data()` usuwa wartości, które powinny być puste (np. datę zakończenia niezakończonych inwestycji).

In [5]:
def fill_address_column() -> None:
    for index in range(len(df.index)):
        if df.at[index, "nr_budynku"] is None:
            full_address: str = f'{df.at[index, "cecha"]} {df.at[index, "ulica_nazwa"]} {df.at[index, "nr_budynku"]}; {df.at[index, "kod_pocztowy"]} {df.at[index, "miejscowosc"]}'
        else:
            full_address: str = f'{df.at[index, "cecha"]} {df.at[index, "ulica_nazwa"]}; {df.at[index, "kod_pocztowy"]} {df.at[index, "miejscowosc"]}'
        df.at[index, "adres"] = full_address


def adjust_aid_received(min_spread: float = 0.65, max_spread: float = 0.85) -> None:
    for index in range(len(df.index)):
        df.at[index, "wartosc_dofinansowania"] = round(
            df.at[index, "wartosc"] * uniform(min_spread, max_spread), 2
        )


def adjust_end_dates() -> None:
    for index in range(len(df.index)):
        if df.at[index, "data_zakonczenia"] < df.at[index, "data_rozpoczecia"]:
            df.at[index, "data_zakonczenia"] = fake.date_between_dates(
                date_start=df.at[index, "data_rozpoczecia"], date_end="-1m"
            )


def fill_popup_url_html() -> None:
    for index in range(len(df.index)):
        output_html: str | None = ""
        links: list[tuple[str | None, str]] = [
            (df.at[index, "url_fiszka"], "Więcej"),
            (df.at[index, "url_geoportal"], "Geoportal"),
            (df.at[index, "url_mapa"], "Mapa"),
        ]
        if links[0][0] is None and links[1][0] is None and links[2][0] is None:
            output_html = None
        else:
            for link in links:
                url, title = link
                if url is not None:
                    output_html += f'<a href="{url}">{title}</a>'
                    output_html = output_html.replace("</a><a", "</a> | <a")
                    if len(output_html) > 256:
                        output_html = None
        df.at[index, "popup_url_html"] = output_html


def clean_unwanted_data() -> None:
    df.loc[df["czy_zakonczony"] == False, "data_zakonczenia"] = None
    df.loc[df["czy_dofinansowany"] == False, "zrodlo_dofinansowania"] = None
    df.loc[df["czy_dofinansowany"] == False, "wartosc_dofinansowania"] = None
    df.loc[df["czy_bo"] == False, "edycja_bo"] = None

In [6]:
df = pd.DataFrame(data=d)
fill_address_column()
adjust_aid_received()
adjust_end_dates()
fill_popup_url_html()
clean_unwanted_data()
df

,foto_sciezka,kategoria,nazwa,zakres,cecha,ulica_nazwa,nr_budynku,kod_pocztowy,miejscowosc,adres,...,zrodlo_dofinansowania,wartosc_dofinansowania,czy_bo,edycja_bo,url_fiszka,url_geoportal,url_mapa,popup_url_html,y,x
0,None,oświata,Przebywać prosić związać mgła film zachodni sł...,Koza teoria pozwalać główny. Wpływ lipiec lęk ...,ul.,Zacisze,None,54-014,Siemianowice Śląskie,ul. Zacisze None; 54-014 Siemianowice Śląskie,...,Wojewódzki Program Naprawy do Poprawy,6944990.35,True,1977,https://example.net/okreslac-obcy,https://example.org/reka-staw-drzwi,https://example.net/budowac-nie-,"<a href=""https://example.net/okreslac-obcy"">Wi...",50.297105,19.054301
1,None,"kultura, sport i rekreacja",Liczyć bydło zapach całość wrócić górski obowi...,Kurs jakość polować ofiara. Ozdobny dlatego cz...,al.,Rzeczna,None,27-993,Siemianowice Śląskie,al. Rzeczna None; 27-993 Siemianowice Śląskie,...,Fundusz Solidności Wykonania Prac Wszelakich,6202976.28,False,None,https://example.net/zero-dwudziesty,None,None,"<a href=""https://example.net/zero-dwudziesty"">...",50.348560,19.056632
2,None,oświata,Tydzień wolny właśnie po masa,Przed zupełnie obcy rejon.\nZgodnie mówić przy...,pl.,Żeglarska,16,67-125,Siemianowice Śląskie,pl. Żeglarska; 67-125 Siemianowice Śląskie,...,Fundusz Poprawy Efektywności Efermerycznej,6669191.31,False,None,None,https://example.org/zakadac-biaoruski,https://example.net/klimat-jedynie-przy,"<a href=""https://example.org/zakadac-biaoruski...",50.326095,19.010665
3,None,dostępność,Popularny duński szklanka przebywać szef,Czasem uważać słyszeć gdyby wydawać. Kaczka Pi...,pl.,Pałacowa,04,11-704,Siemianowice Śląskie,pl. Pałacowa; 11-704 Siemianowice Śląskie,...,Fundusz Solidności Wykonania Prac Wszelakich,2429358.91,False,None,None,https://example.org/rok-hiszpanski,https://example.net/odcien-tytu-sadowy,"<a href=""https://example.org/rok-hiszpanski"">G...",50.308779,19.052806
4,None,infrastruktura,Kawałek ich cecha rodzic czyn nic całkiem ząb,Pragnienie kapelusz ale nieść.\nWysokość droga...,al.,Szpitalna,55/73,50-903,Siemianowice Śląskie,al. Szpitalna; 50-903 Siemianowice Śląskie,...,None,NaN,True,2016,https://example.net/rasa-wielki-zelazo,None,https://example.net/biedny-gruby,"<a href=""https://example.net/rasa-wielki-zelaz...",50.345610,19.029024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,None,infrastruktura,Trwać organ pierwiastek zero o panować sądzić,Niechęć kilka projekt przeciw trzy. 30 strumie...,ul.,Pocztowa,17/17,26-770,Siemianowice Śląskie,ul. Pocztowa; 26-770 Siemianowice Śląskie,...,None,NaN,False,None,https://example.net/dzieo-uderzac-bad,None,https://example.net/moneta-piekny,"<a href=""https://example.net/dzieo-uderzac-bad...",50.321316,19.001845
196,None,"kultura, sport i rekreacja",Miejscowość alkohol wolność europejski Boże Na...,Kot kanał żywy. Teatr zrobić kolejny pomoc chi...,al.,Kazimierza Wielkiego,878,92-013,Siemianowice Śląskie,al. Kazimierza Wielkiego; 92-013 Siemianowice ...,...,Program Ochrony Wód przed Rybami,999680.93,False,None,None,None,https://example.net/gowny-robotnik,"<a href=""https://example.net/gowny-robotnik"">M...",50.306125,18.997848
197,None,oświata,Grecja pani wewnętrzny koło,Letni środa samolot morze. Cierpieć iść przypo...,pl.,Irysowa,20,44-253,Siemianowice Śląskie,pl. Irysowa; 44-253 Siemianowice Śląskie,...,None,NaN,False,None,https://example.net/ozdoba-osio-grzech,https://example.org/sonce-tak-suma,https://example.net/60-stowarzyszenie,"<a href=""https://example.net/ozdoba-osio-grzec...",50.324663,19.051244
198,None,administracja,Stado roślina według więzienie,Teraz dawny hodować zaburzenie. Dzień wystawa ...,al.,Dworska,None,11-986,Siemianowice Śląskie,al. Dworska None; 11-986 Siemianowice Śląskie,...,Fundusz Ochrony Zabytków i Zabitków,2747124.36,False,None,https://example.net/sonce-korzystac,https://example.org/pika-nozna-budowac,https://example.net/materia-wydarzenie,"<a href=""https://example.net/sonce-korzystac"">...",50.28

## Eksport danych testowych do pliku

Jeżeli dane się nie zapisują, sprawdź wartość flagi `dev_mode` w sekcji *Podstawowe ustawienia* - powinna zostać ustawiona, jako `False`.

In [7]:
if not dev_mode:
    df.to_csv("test_data_set.csv", index=False)